In [1]:
import numpy as np
import random
import shoji
import loompy

In [19]:
db = shoji.connect()
db.refdb.human_development_agg._from_loom("/Users/stelin/cytograph/20200702_human_dev/data/Allbrain.agg.loom", fix_expression_dtype=True)

In [20]:
db.refdb

,Contents
human,"0 workspaces, 0 dimensions, 0 tensors"
human_adult,"0 workspaces, 0 dimensions, 0 tensors"
human_adult_agg,"0 workspaces, 2 dimensions, 416 tensors"
human_development,"0 workspaces, 0 dimensions, 0 tensors"
human_development_agg,"0 workspaces, 2 dimensions, 376 tensors"
mouse_adolescent,"0 workspaces, 2 dimensions, 137 tensors"
mouse_adolescent_agg,"0 workspaces, 2 dimensions, 229 tensors"
mouse_development,"0 workspaces, 2 dimensions, 74 tensors"
mouse_development_agg,"0 workspaces, 2 dimensions, 154 tensors"


In [9]:
import h5py
h5 = h5py.File("/Users/stelin/test.h5", "a")

In [11]:
h5.require_group("/")

<HDF5 group "/" (0 members)>

In [14]:
tensor = db.samples.TenX271_1.Expression

In [17]:
np.array((tensor.dtype, tensor.rank, 1 if tensor.jagged else 0) + tensor.dims + (None,) + tensor.shape)

array(['uint16', 2, 0, 'cells', 'genes', None, 2338, 33538], dtype=object)

In [4]:
def merge_config(a, b, path=None):
	"Merge b into a, modifying a in place and returning it"
	if path is None: path = []
	for key in b:
		if key in a:
			if isinstance(a[key], dict) and isinstance(b[key], dict):
				merge_config(a[key], b[key], path + [str(key)])
			elif a[key] == b[key]:
				pass  # same leaf value
			else:
				a[key] = b[key]
		else:
			a[key] = b[key]
	return a


In [8]:
merge_config({"c": 1, "b": 2, "a": 3}, {"b": 7, "a": {"d": 2, "e": 4}})

{'c': 1, 'b': 7, 'a': {'d': 2, 'e': 4}}

In [2]:
db = shoji.connect()
ws = db.samples.TenX204_1

In [5]:
x = ws.Expression[...]

In [6]:
x.shape

(5354, 33538)

In [9]:
db.samples.TenX207_1

Workspace with 0 workspaces, 2 dimensions and 59 tensors:
  cells <Dimension of variable shape>
  genes <Dimension of shape 33538>
  Accession <Tensor Accession dtype='string' dims=('genes',), shape=(33538,)>
  Age <Tensor Age dtype='string' dims=(), shape=()>
  Ambiguous <Tensor Ambiguous dtype='uint16' dims=('cells', 'genes'), shape=(6238, 33538)>
  BpLength <Tensor BpLength dtype='int64' dims=(), shape=()>
  CDNAConc <Tensor CDNAConc dtype='float64' dims=(), shape=()>
  CellConc <Tensor CellConc dtype='int64' dims=(), shape=()>
  CellCycleFraction <Tensor CellCycleFraction dtype='float32' dims=('cells',), shape=(6238,)>
  CellID <Tensor CellID dtype='string' dims=('cells',), shape=(6238,)>
  Chemistry <Tensor Chemistry dtype='string' dims=(), shape=()>
  Chromosome <Tensor Chromosome dtype='string' dims=('genes',), shape=(33538,)>
  Clusters <Tensor Clusters dtype='int64' dims=('cells',), shape=(6238,)>
  Comment <Tensor Comment dtype='string' dims=(), shape=()>
  CreationDate <Tensor CreationDate dtype='string' dims=(), shape=()>
  Donor <Tensor Donor dtype='string' dims=(), shape=()>
  DoubletFinderFlag <Tensor DoubletFinderFlag dtype='int64' dims=('cells',), shape=(6238,)>
  DoubletFinderScore <Tensor DoubletFinderScore dtype='float64' dims=('cells',), shape=(6238,)>
  DoubletFlag <Tensor DoubletFlag dtype='bool' dims=('cells',), shape=(6238,)>
  DoubletScore <Tensor DoubletScore dtype='float32' dims=('cells',), shape=(6238,)>
  End <Tensor End dtype='int64' dims=('genes',), shape=(33538,)>
  Expression <Tensor Expression dtype='uint16' dims=('cells', 'genes'), shape=(6238, 33538)>
  FinalConc <Tensor FinalConc dtype='float64' dims=(), shape=()>
  Gene <Tensor Gene dtype='string' dims=('genes',), shape=(33538,)>
  GeneTotalUMIs <Tensor GeneTotalUMIs dtype='uint32' dims=('genes',), shape=(33538,)>
  LOOM_SPEC_VERSION <Tensor LOOM_SPEC_VERSION dtype='string' dims=(), shape=()>
  MT_ratio <Tensor MT_ratio dtype='float64' dims=('cells',), shape=(6238,)>
  MeanExpression <Tensor MeanExpression dtype='float32' dims=('genes',), shape=(33538,)>
  MeanTotalUMI <Tensor MeanTotalUMI dtype='float64' dims=(), shape=()>
  MitoFraction <Tensor MitoFraction dtype='float32' dims=('cells',), shape=(6238,)>
  NGenes <Tensor NGenes dtype='uint32' dims=('cells',), shape=(6238,)>
  Nonzeros <Tensor Nonzeros dtype='uint32' dims=('genes',), shape=(33538,)>
  PCA <Tensor PCA dtype='float32' dims=('cells', 50), shape=(6238, 50)>
  PCRCycles <Tensor PCRCycles dtype='int64' dims=(), shape=()>
  PassedQC <Tensor PassedQC dtype='bool' dims=(), shape=()>
  PassedQC_MT_ratio <Tensor PassedQC_MT_ratio dtype='uint8' dims=('cells',), shape=(6238,)>
  PassedQC_TotalUMI <Tensor PassedQC_TotalUMI dtype='uint8' dims=('cells',), shape=(6238,)>
  PassedQC_UMIPerGene <Tensor PassedQC_UMIPerGene dtype='uint8' dims=('cells',), shape=(6238,)>
  PassedQC_UnsplicedRatio <Tensor PassedQC_UnsplicedRatio dtype='uint8' dims=('cells',), shape=(6238,)>
  RegionOfInterest <Tensor RegionOfInterest dtype='string' dims=(), shape=()>
  SampleName <Tensor SampleName dtype='string' dims=(), shape=()>
  Sex <Tensor Sex dtype='string' dims=(), shape=()>
  Species <Tensor Species dtype='string' dims=(), shape=()>
  Spliced <Tensor Spliced dtype='uint16' dims=('cells', 'genes'), shape=(6238, 33538)>
  Start <Tensor Start dtype='int64' dims=('genes',), shape=(33538,)>
  StdevExpression <Tensor StdevExpression dtype='float32' dims=('genes',), shape=(33538,)>
  Strand <Tensor Strand dtype='string' dims=('genes',), shape=(33538,)>
  TSNE <Tensor TSNE dtype='float32' dims=('cells', 2), shape=(6238, 2)>
  TargetNumCells <Tensor TargetNumCells dtype='int64' dims=(), shape=()>
  Tissue <Tensor Tissue dtype='string' dims=(), shape=()>
  TotalUMI <Tensor TotalUMI dtype='float64' dims=('cells',), shape=(6238,)>
  TotalUMIs <Tensor TotalUMIs dtype='uint32' dims=('cells',), shape=(6238,)>
  Unspliced <Tensor Unspliced dtype='uint16' dims=('cells', 'genes'), shape=(6238, 33538)>
  UnsplicedFractio

In [18]:
db.builds.humandev.Cerebellum

Workspace with 0 workspaces, 2 dimensions and 23 tensors:
  cells <Dimension of variable shape>
  genes <Dimension of shape 33538>
  Accession <Tensor Accession dtype='string' dims=('genes',), shape=(33538,)>
  Age <Tensor Age dtype='string' dims=('cells',), shape=(940,)>
  CellCycleFraction <Tensor CellCycleFraction dtype='float32' dims=('cells',), shape=(940,)>
  Chemistry <Tensor Chemistry dtype='string' dims=('cells',), shape=(940,)>
  Chromosome <Tensor Chromosome dtype='string' dims=('genes',), shape=(33538,)>
  Donor <Tensor Donor dtype='string' dims=('cells',), shape=(940,)>
  DoubletFlag <Tensor DoubletFlag dtype='bool' dims=('cells',), shape=(940,)>
  DoubletScore <Tensor DoubletScore dtype='float32' dims=('cells',), shape=(940,)>
  End <Tensor End dtype='int64' dims=('genes',), shape=(33538,)>
  Expression <Tensor Expression dtype='uint16' dims=('cells', 'genes'), shape=(940, 33538)>
  Gene <Tensor Gene dtype='string' dims=('genes',), shape=(33538,)>
  MitoFraction <Tensor MitoFraction dtype='float32' dims=('cells',), shape=(940,)>
  NGenes <Tensor NGenes dtype='uint32' dims=('cells',), shape=(940,)>
  SampleName <Tensor SampleName dtype='string' dims=('cells',), shape=(940,)>
  Sex <Tensor Sex dtype='string' dims=('cells',), shape=(940,)>
  Spliced <Tensor Spliced dtype='uint16' dims=('cells', 'genes'), shape=(940, 33538)>
  Start <Tensor Start dtype='int64' dims=('genes',), shape=(33538,)>
  Strand <Tensor Strand dtype='string' dims=('genes',), shape=(33538,)>
  Tissue <Tensor Tissue dtype='string' dims=('cells',), shape=(940,)>
  TotalUMIs <Tensor TotalUMIs dtype='uint32' dims=('cells',), shape=(940,)>
  Unspliced <Tensor Unspliced dtype='uint16' dims=('cells', 'genes'), shape=(940, 33538)>
  UnsplicedFraction <Tensor UnsplicedFraction dtype='float32' dims=('cells',), shape=(940,)>
  ValidCells <Tensor ValidCells dtype='bool' dims=('cells',), shape=(940,)>

In [56]:
ws = db.builds.humandev.Cerebellum
n = ws.TotalUMIs[:]
n_sum = n.sum()
gn = ws.GeneTotalUMIs[:]
d_j = np.zeros((1, ws.genes.length))
for ix in range(0, ws.cells.length, 1000):
    pi_hat_j = (gn / n_sum)[None, :]
    y_ji = ws.Expression[ix: ix + 1000]
    n_i = n[ix: ix + 1000][:, None]
    with np.errstate(divide='ignore', invalid='ignore'):
        # This would be faster as a numba nested for loop
        d_j += 2 * np.sum(np.nan_to_num(y_ji * np.log(div0(y_ji , (n_i * pi_hat_j)))) + (n_i - y_ji) * np.log((n_i - y_ji) / (n_i * (1 - pi_hat_j))), axis=0)

In [57]:
d_j

array([[  0.        , 225.12867581,  65.51292392, ...,   0.        ,
          0.        ,   0.        ]])

In [45]:
def div0(a: np.ndarray, b: np.ndarray) -> np.ndarray:
	""" ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
	with np.errstate(divide='ignore', invalid='ignore'):
		c = np.true_divide(a, b)
		c[~np.isfinite(c)] = 0  # -inf inf NaN
	return c
2 * np.sum(np.nan_to_num(y_ji * np.log(div0(y_ji , (n_i[:, None] * pi_hat_j[None, :])))) + (n_i[:, None] - y_ji) * np.log((n_i[:, None] - y_ji) / (n_i[:, None] * (1 - pi_hat_j[None, :]))), axis=0)

<ipython-input-45-8967f57f6685>:7: RuntimeWarning: divide by zero encountered in log
  2 * np.sum(np.nan_to_num(y_ji * np.log(div0(y_ji , (n_i[:, None] * pi_hat_j[None, :])))) + (n_i[:, None] - y_ji) * np.log((n_i[:, None] - y_ji) / (n_i[:, None] * (1 - pi_hat_j[None, :]))), axis=0)
<ipython-input-45-8967f57f6685>:7: RuntimeWarning: invalid value encountered in multiply
  2 * np.sum(np.nan_to_num(y_ji * np.log(div0(y_ji , (n_i[:, None] * pi_hat_j[None, :])))) + (n_i[:, None] - y_ji) * np.log((n_i[:, None] - y_ji) / (n_i[:, None] * (1 - pi_hat_j[None, :]))), axis=0)


array([0.        , 2.08423978, 0.49040934, ..., 0.        , 0.        ,
       0.        ])

In [41]:
(n_i[:, None] - y_ji)

array([[12784, 12784, 12784, ..., 12784, 12784, 12784],
       [13307, 13307, 13307, ..., 13307, 13307, 13307],
       [ 8177,  8177,  8177, ...,  8177,  8177,  8177],
       ...,
       [20886, 20886, 20886, ..., 20886, 20886, 20886],
       [12793, 12793, 12793, ..., 12793, 12793, 12793],
       [11792, 11792, 11792, ..., 11792, 11792, 11792]], dtype=uint32)

In [43]:
np.log((n_i[:, None] - y_ji) / (n_i[:, None] * (1 - pi_hat_j[None, :])))

array([[0.00000000e+00, 9.10581797e-08, 2.14254532e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.10581797e-08, 2.14254534e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.10581797e-08, 2.14254534e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 9.10581795e-08, 2.14254534e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.10581797e-08, 2.14254532e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.10581797e-08, 2.14254532e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [31]:
gn

array([ 0, 17,  4, ...,  0,  0,  0], dtype=uint32)